In [1]:
import sys
sys.path.insert(0, "../src/gen")
sys.path.insert(1, "../src/rte")
from pathlib import Path
from collections import Counter

import numpy as np
from scipy.special import softmax

from util import read_data, write_jsonl
from aggregate import generate_micro_macro_df, generate_doc_df

In [2]:
# constants
LOOKUP = {
    "verifiable": {"no": "NOT VERIFIABLE", "yes": "VERIFIABLE"},
    "label": {"nei": "NOT ENOUGH INFO", "r": "REFUTES", "s": "SUPPORTS"}
}

SEED = 123456789

LABEL2ID = {"SUPPORTS": 0, "NOT ENOUGH INFO": 1, "REFUTES": 2}
ID2LABEL = {0: "SUPPORTS", 1: "NOT ENOUGH INFO", 2: "REFUTES"}

# Init

In [3]:
import evaluate
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer,
    DataCollatorWithPadding,
    TextClassificationPipeline,
    pipeline
)

import torch
torch.backends.cuda.matmul.allow_tf32 = True

# Huggingface Init

## Model

In [4]:
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

In [5]:
model_checkpoint = "xlnet-base-cased"
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint, 
        num_labels=3, 
        id2label=ID2LABEL, 
        label2id=LABEL2ID
    )

model = model_init()
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, do_lower_case=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess(examples):
    return tokenizer(examples["evidence"], examples["claim"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    results = {}
    results.update(accuracy_metric.compute(predictions=predictions, references=labels))
    results.update(recall_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(precision_metric.compute(predictions=predictions, references=labels, average="macro"))
    results.update(f1_metric.compute(predictions=predictions, references=labels, average="macro"))
    
    return results

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [6]:
dataset = ["fever", "climatefeverpure", "fever-climatefeverpure", "climatefever", "fever-climatefever"]
task = ["hp_tune", "output"]
doc_sent = ["doc", "sent"]

di = 2
ti = 0
ds = 0

model_store_path = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/thesis/models").joinpath(model_checkpoint)
model_store_path.mkdir(exist_ok=True)
model_store_path = model_store_path / f"{dataset[di]}-{model_checkpoint}-{doc_sent[ds]}-{task[ti]}"

## Dataset

In [7]:
datap = Path(f"/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/bert-data-{doc_sent[ds]}-evidence")

data = DatasetDict({
    "train": Dataset.from_list(read_data(datap / f"{dataset[di]}.train.n5.jsonl")),
    "validation": Dataset.from_list(read_data(datap / f"{dataset[di]}.dev.n5.jsonl")),
    "fever_test": Dataset.from_list(read_data(datap / f"{dataset[0]}.test.n5.jsonl")),
    "climatefever_test": Dataset.from_list(read_data(datap / f"{dataset[1]}.test.n5.jsonl"))
}).map(preprocess, batched=True)

Map:   0%|          | 0/146415 [00:00<?, ? examples/s]

Map:   0%|          | 0/10277 [00:00<?, ? examples/s]

Map:   0%|          | 0/9999 [00:00<?, ? examples/s]

Map:   0%|          | 0/137 [00:00<?, ? examples/s]

## Trainer

In [8]:
# effective batch size of 32
per_device_train_batch_size = 4
gradient_accumulation_steps = 8
per_device_eval_batch_size = 32


learning_rate = 4e-4
epoch = 4
metric_name = "f1"
warmup_ratio=0.1
save_steps=200
eval_steps=200

# Hyperparameter tuning

In [9]:
# shard the data if the dataset is large for hyperparameter tuning
shard = data["train"].num_rows > 50000
hp_tune_train = data["train"].shuffle(seed=SEED).shard(num_shards=5, index=1)

In [10]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=eval_steps,
    save_strategy = "no",
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard",
    tf32=True
)

_ = model.train()
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=hp_tune_train if shard else data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [11]:
def optuna_hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [1e-5, 3e-5, 2e-5]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 1, 5),
    }

def compute_objective(metrics):
    return metrics["f1"]

In [12]:
best_run = trainer.hyperparameter_search(
    direction="maximize", 
    n_trials=10, 
    hp_space=optuna_hp_space
)

[I 2023-07-04 13:33:37,304] A new study created in memory with name: no-name-dcbe787d-f66a-4bd2-af76-b8a98d4d3867
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias', 'logi

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.378833,0.837307,0.836011,0.855487,0.832568
400,No log,0.399713,0.863482,0.862281,0.885130,0.860991
600,0.452900,0.275815,0.909507,0.909022,0.912236,0.908609
800,0.452900,0.235704,0.916318,0.915778,0.921890,0.916091
1000,0.219200,0.277892,0.921962,0.921714,0.922702,0.921687
1200,0.219200,0.260226,0.919529,0.918989,0.924739,0.918920
1400,0.219200,0.232552,0.926535,0.926139,0.930207,0.926628
1600,0.174500,0.258453,0.924978,0.924559,0.928125,0.924700
1800,0.174500,0.230073,0.927897,0.927676,0.929492,0.928107
2000,0.144800,0.285133,0.928773,0.928547,0.930368,0.928993


[I 2023-07-04 14:20:15,654] Trial 0 finished with value: 3.7242138598056367 and parameters: {'learning_rate': 3e-05, 'num_train_epochs': 5}. Best is trial 0 with value: 3.7242138598056367.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.461152,0.791379,0.789335,0.832482,0.775212
400,No log,0.341258,0.891603,0.891080,0.895289,0.890827
600,0.441000,0.289134,0.897344,0.896626,0.905870,0.896682
800,0.441000,0.286995,0.897538,0.896639,0.911930,0.896698
1000,0.216500,0.279071,0.920016,0.919603,0.923609,0.920059
1200,0.216500,0.222393,0.928384,0.928012,0.930923,0.928179
1400,0.216500,0.232925,0.920989,0.920545,0.925446,0.921088
1600,0.168700,0.245319,0.924297,0.923877,0.928008,0.924271
1800,0.168700,0.217089,0.932471,0.932189,0.933861,0.932297
2000,0.136200,0.293239,0.926146,0.925934,0.927853,0.926438


[I 2023-07-04 14:57:40,601] Trial 1 finished with value: 3.7221037008785203 and parameters: {'learning_rate': 3e-05, 'num_train_epochs': 4}. Best is trial 0 with value: 3.7242138598056367.
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.

...

[I 2023-07-04 16:04:12,811] Trial 7 pruned. 
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-strea

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.654347,0.647271,0.643296,0.820709,0.539004


[I 2023-07-04 16:06:16,679] Trial 8 pruned. 
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-strea

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.468949,0.780383,0.778189,0.830208,0.760902


[I 2023-07-04 16:08:20,431] Trial 9 pruned. 


In [13]:
best_run

BestRun(run_id='0', objective=3.7242138598056367, hyperparameters={'learning_rate': 3e-05, 'num_train_epochs': 5}, run_summary=None)

## Train with best hyperparameters

In [14]:
training_args = TrainingArguments(
    model_store_path,
    overwrite_output_dir=True,
    evaluation_strategy = "steps",
    eval_steps=eval_steps,
    save_strategy="steps",
    save_steps=save_steps,
    save_total_limit=5,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=epoch,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=False,
    seed=SEED,
    data_seed=SEED,
    warmup_ratio=warmup_ratio,
    report_to="tensorboard",
    tf32=True
)

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)
    
trainer.train()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
200,No log,0.752081,0.632675,0.628747,0.475035,0.524713
400,No log,0.409956,0.819694,0.817939,0.858141,0.810736
600,0.615500,0.270530,0.892089,0.891395,0.900663,0.891751
800,0.615500,0.320059,0.895689,0.894779,0.907746,0.893785
1000,0.249100,0.234248,0.923227,0.922905,0.925200,0.923208
1200,0.249100,0.306150,0.914177,0.913644,0.919202,0.913881
1400,0.249100,0.315622,0.915248,0.914649,0.922489,0.915227
1600,0.215600,0.280649,0.910966,0.910295,0.919092,0.910506
1800,0.215600,0.239257,0.921767,0.921898,0.923153,0.922120
2000,0.203100,0.320043,0.899582,0.898852,0.910636,0.899763


/scratch/users/k21190024/envs/conda/p-dis-torch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=22875, training_loss=0.12231927673673369, metrics={'train_runtime': 14282.6431, 'train_samples_per_second': 51.256, 'train_steps_per_second': 1.602, 'total_flos': 5.552116534613016e+16, 'train_loss': 0.12231927673673369, 'epoch': 5.0})

In [15]:
trainer.save_model(model_store_path.parent / (model_store_path.stem + ".out"))

# Evaluate

In [16]:
import pandas as pd
from sklearn.metrics import classification_report

## Test on validation data

In [17]:
preds = trainer.predict(data["validation"])
val = generate_doc_df(data["validation"], preds)

In [18]:
print(classification_report(y_true=val["actual"], y_pred=val["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       1.00      0.98      0.99      3428
        REFUTES       0.95      0.90      0.92      3384
       SUPPORTS       0.90      0.95      0.93      3465

       accuracy                           0.95     10277
      macro avg       0.95      0.95      0.95     10277
   weighted avg       0.95      0.95      0.95     10277



## Test on test data

### FEVER

In [19]:
preds = trainer.predict(data["fever_test"])
ftes = generate_doc_df(data["fever_test"], preds)

In [20]:
print(classification_report(y_true=ftes["actual"], y_pred=ftes["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       1.00      0.99      1.00      3333
        REFUTES       0.95      0.88      0.92      3333
       SUPPORTS       0.89      0.96      0.92      3333

       accuracy                           0.95      9999
      macro avg       0.95      0.95      0.95      9999
   weighted avg       0.95      0.95      0.95      9999



### Climate-FEVER

In [21]:
preds = trainer.predict(data["climatefever_test"])
cftes = generate_doc_df(data["climatefever_test"], preds)

In [22]:
print(classification_report(y_true=cftes["actual"], y_pred=cftes["predicted"]))

                 precision    recall  f1-score   support

NOT ENOUGH INFO       0.91      0.66      0.77        47
        REFUTES       0.48      0.60      0.54        25
       SUPPORTS       0.71      0.78      0.74        65

       accuracy                           0.71       137
      macro avg       0.70      0.68      0.68       137
   weighted avg       0.74      0.71      0.71       137

